In [1]:
import fsspec_reference_maker
fsspec_reference_maker.__version__

'0.0.3'

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
from fsspec_reference_maker.hdf import SingleHdf5ToZarr
from fsspec_reference_maker.combine import MultiZarrToZarr
import fsspec

/home/lsterzinger/anaconda3/lib/python3.7/site-packages/xarray/backends/cfgrib_.py:28: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  "Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. "


In [3]:
fs = fsspec.filesystem('s3', anon=True)

In [4]:
flist = fs.glob("s3://noaa-goes16/ABI-L2-SSTF/2020/210/*/*.nc")

In [6]:
flist = ['s3://' + f for f in flist]

In [8]:
from dask.distributed import Client
client = Client()
client

/home/lsterzinger/anaconda3/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35117 instead
  f"Port {expected} is already in use.\n"
distributed.diskutils - INFO - Found stale lock file and directory '/home/lsterzinger/Desktop/ncar-earth-system-data-science-wip/dask-worker-space/worker-a46k_d6m', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/lsterzinger/Desktop/ncar-earth-system-data-science-wip/dask-worker-space/worker-5k4mvy34', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/lsterzinger/Desktop/ncar-earth-system-data-science-wip/dask-worker-space/worker-ke3j3ihx', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/lsterzinger/Desktop/ncar-earth-system-data-science-wip/dask-worker-space/worker-cwvuc2rb', purging
distributed.diskutils - INFO - Found stale lock file 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:35117/status,
Dashboard: http://127.0.0.1:35117/status,Workers: 4
Total threads: 8,Total memory: 15.56 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40235,Workers: 4
Dashboard: http://127.0.0.1:35117/status,Total threads: 8
Started: Just now,Total memory: 15.56 GiB
Comm: tcp://127.0.0.1:34667,Total threads: 2
Dashboard: http://127.0.0.1:42561/status,Memory: 3.89 GiB
Nanny: tcp://127.0.0.1:42771,


In [9]:
import dask
import dask.bag as db


In [11]:
b = db.from_sequence(flist)
b

dask.bag<from_sequence, npartitions=24>

In [12]:

def gen_ref(f):
    
    with fs.open(f, 'rb') as infile:
        return SingleHdf5ToZarr(infile, f, inline_threshold=300).translate()

    

In [13]:
dicts = b.map(gen_ref).compute()

distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:44675
Traceback (most recent call last):
  File "/home/lsterzinger/anaconda3/lib/python3.7/site-packages/distributed/comm/core.py", line 318, in connect
    handshake = await asyncio.wait_for(comm.read(), time_left())
  File "/home/lsterzinger/anaconda3/lib/python3.7/asyncio/tasks.py", line 449, in wait_for
    raise futures.TimeoutError()
concurrent.futures._base.TimeoutError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/lsterzinger/anaconda3/lib/python3.7/site-packages/distributed/worker.py", line 2374, in gather_dep
    self.rpc, to_gather_keys, worker, who=self.address
  File "/home/lsterzinger/anaconda3/lib/python3.7/site-packages/distributed/worker.py", line 3744, in get_data_from_worker
    return await retry_operation(_get_data, operation="get_data_from_worker")
  File "/home/lsterzinger/anaconda3/lib/python3.7/site-pack

In [27]:
mzz = MultiZarrToZarr(
    dicts[1:],
    remote_protocol='s3',
    remote_options={'anon':True},
    xarray_concat_args={'dim' : 't'}
)

In [28]:
mzz_dict = mzz.translate()

ValueError: 'max_reflectance_factor' is not present in all datasets.

In [21]:
fs2 = fsspec.filesystem('reference', fo=mzz_dict, remote_protocol='s3', remote_options=dict(anon=True))
ds = xr.open_dataset(fs2.get_mapper(""), engine='zarr')
ds

/home/lsterzinger/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  


ValueError: cannot reshape array of size 2 into shape (2,2)